In [ ]:
# feature extraction of normal (also same for stroke)
import os
import pandas as pd
import numpy as np
from pathlib import Path
from scipy.signal import welch
from scipy.stats import kurtosis, skew
import pywt  # For wavelet features

# New channel groups and features per the mapping
channels_features_map = {
    "FP1": ["mean", "var", "hjorth", "approx_entropy", "alpha_beta_ratio"],
    "FP2": ["mean", "var", "hjorth", "approx_entropy", "alpha_beta_ratio"],

    "FC3": ["line_length", "ssc", "zcr", "alpha_beta_power", "wavelet_energy", "entropy"],
    "FC4": ["line_length", "ssc", "zcr", "alpha_beta_power", "wavelet_energy", "entropy"],
    "FCZ": ["line_length", "ssc", "zcr", "alpha_beta_power", "wavelet_energy", "entropy"],

    "F3": ["var", "rms", "ptp", "theta_beta_ratio", "spectral_entropy"],
    "F4": ["var", "rms", "ptp", "theta_beta_ratio", "spectral_entropy"],
    "FZ": ["var", "rms", "ptp", "theta_beta_ratio", "spectral_entropy"],

    "C3": ["hjorth", "psd_delta_theta", "line_length", "delta_theta_ratio"],
    "C4": ["hjorth", "psd_delta_theta", "line_length", "delta_theta_ratio"],
    "CZ": ["hjorth", "psd_delta_theta", "line_length", "delta_theta_ratio"],

    "CP3": ["sample_entropy", "ptp", "delta_theta_ratio", "wavelet_coefficients", "entropy"],
    "CP4": ["sample_entropy", "ptp", "delta_theta_ratio", "wavelet_coefficients", "entropy"],
    "CPZ": ["sample_entropy", "ptp", "delta_theta_ratio", "wavelet_coefficients", "entropy"],
}

BANDS = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30)
}

fs = 160  # Sampling frequency (Hz)


def compute_bandpower(signal, fs, band):
    fmin, fmax = band
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2)
    mask = (freqs >= fmin) & (freqs <= fmax)
    power = np.trapz(psd[mask], freqs[mask])
    return power


def line_length(signal):
    return np.sum(np.abs(np.diff(signal)))


def slope_sign_changes(signal):
    diff_signal = np.diff(signal)
    diff_sign = np.sign(diff_signal)
    ssc = np.sum(diff_sign[:-1] != diff_sign[1:])
    return ssc


def zero_crossing_rate(signal):
    return np.sum((signal[:-1] * signal[1:]) < 0)


def peak_to_peak(signal):
    return np.ptp(signal)


def spectral_entropy(signal):
    freqs, psd = welch(signal, fs=fs, nperseg=fs*2)
    psd_norm = psd / np.sum(psd)
    psd_norm = psd_norm[psd_norm > 0]  # Avoid log(0)
    entropy = -np.sum(psd_norm * np.log2(psd_norm))
    return entropy


def approximate_entropy(signal, m=2, r=None):
    # Approximate entropy calculation simplified version
    N = len(signal)
    if r is None:
        r = 0.2 * np.std(signal)
    def _phi(m):
        x = np.array([signal[i:i + m] for i in range(N - m + 1)])
        C = np.sum(np.max(np.abs(x[:, None] - x[None, :]), axis=2) <= r, axis=0) / (N - m + 1)
        return np.sum(np.log(C)) / (N - m + 1)
    return abs(_phi(m) - _phi(m + 1))


def sample_entropy(signal, m=2, r=None):
    # Sample entropy approximation
    N = len(signal)
    if r is None:
        r = 0.2 * np.std(signal)

    def _count_similar(seg_len):
        x = np.array([signal[i:i + seg_len] for i in range(N - seg_len + 1)])
        count = 0
        total = 0
        for i in range(len(x)):
            dist = np.max(np.abs(x - x[i]), axis=1)
            count += np.sum(dist <= r) - 1
            total += len(dist) - 1
        return count / total if total > 0 else 0

    B = _count_similar(m)
    A = _count_similar(m + 1)
    if B == 0:
        return np.inf
    if A == 0:
        return -np.log(1.0 / (N - m))
    return -np.log(A / B)


def wavelet_energy(signal, wavelet='db4', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    energy = np.sum([np.sum(c ** 2) for c in coeffs])
    return energy


def wavelet_coefficients_stats(signal, wavelet='db4', level=4):
    coeffs = pywt.wavedec(signal, wavelet, level=level)
    stats = {}
    for i, c in enumerate(coeffs):
        stats[f"wavelet_coeff_mean_level_{i}"] = np.mean(c)
        stats[f"wavelet_coeff_std_level_{i}"] = np.std(c)
    return stats


def hjorth_parameters(signal):
    diff_signal = np.diff(signal)
    diff_diff_signal = np.diff(diff_signal)
    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_signal) / activity) if activity != 0 else 0
    complexity = np.sqrt(np.var(diff_diff_signal) / np.var(diff_signal)) if np.var(diff_signal) != 0 else 0
    return activity, mobility, complexity


def extract_features_from_trial(trial_df):
    features = {}
    for ch, feats in channels_features_map.items():
        if ch not in trial_df.columns:
            continue
        signal = trial_df[ch].values
        
        # Precompute band power for some features
        bandpowers = {band: compute_bandpower(signal, fs, band_range) for band, band_range in BANDS.items()}

        # Helper ratios
        alpha_beta_ratio = bandpowers["alpha"] / bandpowers["beta"] if bandpowers["beta"] != 0 else 0
        theta_beta_ratio = bandpowers["theta"] / bandpowers["beta"] if bandpowers["beta"] != 0 else 0
        delta_theta_ratio = bandpowers["delta"] / bandpowers["theta"] if bandpowers["theta"] != 0 else 0

        for feat in feats:
            if feat == "mean":
                features[f"{ch}_mean"] = np.mean(signal)
            elif feat == "var":
                features[f"{ch}_var"] = np.var(signal)
            elif feat == "rms":
                features[f"{ch}_rms"] = np.sqrt(np.mean(signal ** 2))
            elif feat == "skew":
                features[f"{ch}_skew"] = skew(signal)
            elif feat == "kurtosis":
                features[f"{ch}_kurtosis"] = kurtosis(signal)
            elif feat == "approx_entropy":
                features[f"{ch}_approx_entropy"] = approximate_entropy(signal)
            elif feat == "sample_entropy":
                features[f"{ch}_sample_entropy"] = sample_entropy(signal)
            elif feat == "line_length":
                features[f"{ch}_line_length"] = line_length(signal)
            elif feat == "ssc":
                features[f"{ch}_ssc"] = slope_sign_changes(signal)
            elif feat == "zcr":
                features[f"{ch}_zero_crossings"] = zero_crossing_rate(signal)
            elif feat == "peak_to_peak" or feat == "ptp":
                features[f"{ch}_peak_to_peak"] = peak_to_peak(signal)
            elif feat == "hjorth":
                activity, mobility, complexity = hjorth_parameters(signal)
                features[f"{ch}_hjorth_activity"] = activity
                features[f"{ch}_hjorth_mobility"] = mobility
                features[f"{ch}_hjorth_complexity"] = complexity
            elif feat == "alpha_beta_ratio":
                features[f"{ch}_alpha_beta_ratio"] = alpha_beta_ratio
            elif feat == "theta_beta_ratio":
                features[f"{ch}_theta_beta_ratio"] = theta_beta_ratio
            elif feat == "delta_theta_ratio":
                features[f"{ch}_delta_theta_ratio"] = delta_theta_ratio
            elif feat == "alpha_beta_power":
                features[f"{ch}_alpha_power"] = bandpowers["alpha"]
                features[f"{ch}_beta_power"] = bandpowers["beta"]
                features[f"{ch}_alpha_beta_ratio"] = alpha_beta_ratio
            elif feat == "psd_delta_theta":
                features[f"{ch}_delta_power"] = bandpowers["delta"]
                features[f"{ch}_theta_power"] = bandpowers["theta"]
                features[f"{ch}_delta_theta_ratio"] = delta_theta_ratio
            elif feat == "spectral_entropy":
                features[f"{ch}_spectral_entropy"] = spectral_entropy(signal)
            elif feat == "entropy":
                # compute histogram entropy as previously (approximate)
                probs, _ = np.histogram(signal, bins=30, density=True)
                probs = probs[probs > 0]
                entropy_val = -np.sum(probs * np.log2(probs))
                features[f"{ch}_entropy"] = entropy_val
            elif feat == "wavelet_energy":
                features[f"{ch}_wavelet_energy"] = wavelet_energy(signal)
            elif feat == "wavelet_coefficients":
                wavelet_stats = wavelet_coefficients_stats(signal)
                for key, val in wavelet_stats.items():
                    features[f"{ch}_{key}"] = val

    return features




input_root = Path(r"D:\ai\eeg_data\nromal\preprocessed_trials\normal")
output_root = Path(r"D:\ai\eeg_data\nromal\normal_feature")
output_root.mkdir(parents=True, exist_ok=True)

normal_subjects = [f"S{i:03}" for i in range(1, 110)]  # S001 to S109

for subj in normal_subjects:
    print(f" Extracting features for NORMAL: {subj}")
    subj_folder = input_root / subj
    output_file = output_root / f"{subj}_features.csv"

    all_features = []

    if not subj_folder.exists():
        print(f" Missing subject folder: {subj_folder}")
        continue

    for trial_file in subj_folder.glob("trial_*.csv"):
        trial_df = pd.read_csv(trial_file)

        # Basic info
        meta = {
            "trial_id": trial_df["trial_id"].iloc[0],
            "label": trial_df["label"].iloc[0],
            "onset_sec": trial_df["onset_sec"].iloc[0],
            "duration_sec": trial_df["duration_sec"].iloc[0]
        }

        feat = extract_features_from_trial(trial_df)   # <--- Reuse your existing function here
        all_features.append({**meta, **feat})

    if all_features:
        df_out = pd.DataFrame(all_features)
        df_out.to_csv(output_file, index=False)
        print(f" Saved: {output_file}")
    else:
        print(f" No trials found for {subj}")
